In [3]:
from pybaselines import Baseline, utils
import pandas as pd
import spectral
import pysptools
from spectral import *
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal, interpolate, stats
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
import pickle

from scipy.optimize import nnls
from numpy.linalg import norm
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import glob

In [31]:
def compute_EGO_Gaussian(lmbda, si, mu_i, sigma_i, k_i):
    # EGO Gaussian function
    return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))

def weighted_sum_EGO_Gaussians(x, *params):
    num_peaks = len(params) // 4
    result = np.zeros_like(x)
    for i in range(num_peaks):
        si, mu_i, sigma_i, k_i = params[i*4 : (i+1)*4]
        result += compute_EGO_Gaussian(x, si, mu_i, sigma_i, k_i)
    return result

def find_initial_guess(x, y):
    peaks, _ = find_peaks(y, width=5)
    initial_guess = []
    for p in peaks:
        initial_guess += [1.0, x[p], 10, -0.1]
    return initial_guess

# Widget to select a file from a list of pickled files in the current directory
pickle_files = glob.glob('*.pickle')
file_widget = widgets.Dropdown(options=pickle_files, description='Select file:')

# Function to open, load and close a pickled file
def load_data(file_name):
    with open(file_name, 'rb') as file:
        data = pickle.load(file)
    return data

# Widget to set the current spectrum in X and Y coordinates
x_widget = widgets.IntSlider(min=0,
                             max=100,
                             description='Current X Pixel:')
y_widget = widgets.IntSlider(min=0,
                             max=100,
                             description='Current Y Pixel:')

# Widget to set the lower and upper wavelength cutoff
start_widget = widgets.IntSlider(min=0,
                                 max=512,
                                 description='Start:')
end_widget = widgets.IntSlider(min=0,
                               max=512,
                               value = 512,
                               description='End:')

# Widget to control the width parameter
width_widget = widgets.IntSlider(min=0,
                                 max=100,
                                 value = 5,
                                 description='Peak Width Threshold:')

def update_x_range(*args):
    baseline_corrected_data = load_data(file_widget.value)
    # x_widget.max = baseline_corrected_data.shape[0]
    # start_widget.max = baseline_corrected_data.shape[1]
    # end_widget.max = baseline_corrected_data.shape[1]
    # end_widget.value = baseline_corrected_data.shape[1]
    
file_widget.observe(update_x_range, 'value')

@interact(file=file_widget, curr_x=x_widget, curr_y=y_widget, curr_start=start_widget, curr_end=end_widget, width=width_widget)
def interactive_plot(file, curr_x, curr_y, curr_start, curr_end, width):
    baseline_corrected_data = load_data(file)

    curr_spectrum = baseline_corrected_data[curr_x,curr_y,:]

    curr_peak = curr_spectrum[curr_start:curr_end]
    curr_peak = curr_peak / curr_peak.max()
    curr_peak = 1 - curr_peak

    # Handle temp value for wav_array
    temp_used = False
    if 'wav_array' in locals() and temp_used==False:
        curr_wavs = wav_array[curr_start:curr_end]
        temp_used = True
    else:
        curr_wavs = np.linspace(0,curr_peak.shape[0]-1,curr_peak.shape[0])

    test_peaks = find_peaks(curr_peak, width=width)[0]
    plt.plot(curr_wavs, curr_peak)
    plt.plot(curr_wavs[test_peaks], curr_peak[test_peaks], 'o')
    plt.show()

    # Find initial guess for the parameters
    initial_guess = find_initial_guess(curr_wavs,
                                       curr_peak)

    try:
        # Fit the weighted sum of EGO Gaussians to the data
        params, _ = curve_fit(weighted_sum_EGO_Gaussians, curr_wavs, curr_peak, p0=initial_guess)
        
        # Generate x-values for plotting the fitted curve
        # x_values = np.linspace(min(curr_wavs), max(curr_wavs), 100)
        
        # Generate y-values for the fitted curve using the obtained parameters
        y_values = weighted_sum_EGO_Gaussians(curr_wavs, *params)
        
        # calculate r_squared and SSE
        residuals = curr_peak - y_values
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((curr_peak-np.mean(curr_peak))**2)
        r_squared = 1 - (ss_res / ss_tot)
        sse = np.sum((curr_peak - y_values)**2)

        plt.plot(curr_wavs, curr_peak)
        plt.plot(curr_wavs, y_values , 'r-')
        # add the metrics to the plot
        plt.figtext(0.15, 0.85, 'R^2: {:.4f}'.format(r_squared))
        plt.figtext(0.15, 0.80, 'SSE: {:.4f}'.format(sse))

        # Calculate the peak dataframe
        mus = params[np.arange(0,
                         params.shape[0],
                         4)]
        amps = params[np.arange(1,
                              params.shape[0],
                              4)]
        sigmas = params[np.arange(2,
                              params.shape[0],
                              4)]
        asymms = params[np.arange(3,
                              params.shape[0],
                              4)]
        
        curr_df = pd.DataFrame([mus, amps, sigmas, asymms]).T
        curr_df.columns = ['Amplitude', 'Center', 'Width', 'Asymmetry']
        # curr_df['FWHM'] = 2* curr_df.Width * np.sqrt(2*np.log(2))

        for _, row in curr_df.iterrows():
            plt.vlines(x=row['Center'],
                       ymin=0,
                       ymax=row['Amplitude'],
                       colors='g',
                       linestyles='dotted')
            plt.hlines(y=row['Amplitude'] * 0.6,
                       xmin=row['Center'] - row['Width']/2,
                       xmax=row['Center'] + row['Width']/2,
                       colors='g',
                       linestyles='dotted')
    
        plt.show()

        # Show the current fits
        print("Peaks were found in the following locations")

        display(curr_df)
        
        
    except RuntimeError:
        display("Optimization failed with the current parameters, try changing the fit parameters")




interactive(children=(Dropdown(description='Select file:', options=('baseline_corrected_array.pickle', 'baseli…

In [ ]:
curr_x